In [79]:
import numpy as np 
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn.impute import KNNImputer  
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

### 1. KNN Imputation and Classification


#### 1

In [59]:
train = pd.read_csv("DodgerLoopGame_TRAIN.txt", header=None, sep='\s+')
test = pd.read_csv("DodgerLoopGame_TEST.txt", header=None, sep='\s+')

trainNaN = train.isna().sum().sum()
testNaN = test.isna().sum().sum()

print(f"Number of NaN values in the training set: {trainNaN}")
print(f"Number of NaN values in the testing set: {testNaN}")

Number of NaN values in the training set: 65
Number of NaN values in the testing set: 272


#### 2

In [100]:
dummy = train.dropna()  
np.random.seed(42)

# Create a mask to introduce missing values in a COPY of dummy
mask = np.random.rand(*dummy.shape) < 0.2  # Mask 20% of the data
dummy_masked = dummy.mask(mask)  # Introduce NaN values in the copy

# Split masked data into train/val sets
train_data_values = dummy_masked.iloc[:, 1:]  # Exclude the label column
train_labels = dummy_masked.iloc[:, 0]
X_train, X_val_masked = train_test_split(train_data_values, test_size=0.2, random_state=42)

# Create a clean version of X_val from the original dataset (without NaNs)
_, X_val_clean = train_test_split(dummy.iloc[:, 1:], test_size=0.2, random_state=42)


optimal_k = None
min_error = float('inf')

for k in range(2,8):
    imputer = KNNImputer(n_neighbors=k)    
    X_train_imputed = imputer.fit_transform(X_train)
    X_val_imputed = imputer.transform(X_val_masked)
    error = mean_squared_error(X_val_clean, X_val_imputed)
    print(k)
    print(error)
    if error < min_error:
        min_error = error
        optimal_k = k

print(f"Optimal K for imputation: {optimal_k}")

2
9.479817708333334
3
9.939718364197532
4
10.52468532986111
5
10.84642361111111
6
11.785035686728397
7
11.847576530612246
Optimal K for imputation: 2


In [101]:
imputer = KNNImputer(n_neighbors=optimal_k)
train_data_imputed = imputer.fit_transform(train.iloc[:, 1:])
test_data_imputed = imputer.fit_transform(test.iloc[:, 1:])

#### 3

In [61]:
class KNNClassifier:
    def __init__(self, K):
        self.K = K
        self.X_train = None
        self.y_train = None
    
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
    
    def predict(self, X_query):
        predictions = np.zeros(X_query.shape[0], dtype=object)
        
        for i, query_point in enumerate(X_query):
            distances = np.array([euclidean_distance(query_point, x) for x in self.X_train])
            
            nearest_neighbor_indices = np.argsort(distances)[:self.K]
            
            nearest_labels = self.y_train.iloc[nearest_neighbor_indices].values
            
            class_counts = {}
            for label in nearest_labels:
                if label in class_counts:
                    class_counts[label] += 1
                else:
                    class_counts[label] = 1
            
            predicted_class = max(class_counts, key=class_counts.get)
            predictions[i] = predicted_class
        
        return predictions


In [62]:
def euclidean_distance(xa, xb):
    return np.sqrt(np.sum((xa - xb) ** 2))

In [63]:
def accuracy_score(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

In [105]:
x_train = train_data_imputed
x_test = test_data_imputed
y_train = train.iloc[:, 0]
y_test = test.iloc[:, 0]

In [107]:
X_train, X_val, y_train_split, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

best_k = None
best_accuracy = 0

for k in range(1,9):
    knn = KNNClassifier(K=k)
    knn.fit(X_train, y_train_split)
    
    y_val_pred = knn.predict(X_val)
    
    val_accuracy = accuracy_score(y_val, y_val_pred)
    print(f"K={k}, Validation Accuracy={val_accuracy:.4f}")
    
    if val_accuracy > best_accuracy:
        best_k = k
        best_accuracy = val_accuracy

print(f"Optimal K: {best_k} with Validation Accuracy: {best_accuracy:.4f}")

K=1, Validation Accuracy=0.5000
K=2, Validation Accuracy=0.5000
K=3, Validation Accuracy=0.7500
K=4, Validation Accuracy=0.7500
K=5, Validation Accuracy=0.7500
K=6, Validation Accuracy=0.7500
K=7, Validation Accuracy=0.7500
K=8, Validation Accuracy=0.5000
Optimal K: 3 with Validation Accuracy: 0.7500


In [108]:
final_knn = KNNClassifier(K=best_k)
final_knn.fit(x_train, y_train)
y_test_pred = final_knn.predict(x_test)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Test Accuracy with K={best_k}: {test_accuracy:.4f}")

Test Accuracy with K=3: 0.8406


### 2. Decision Tree Regression Implementation

In [23]:
class TreeNode:
    def __init__(self):
        self.predicted_value = None
        self.split_feature = None
        self.threshold = None
        self.left = None
        self.right = None

In [91]:
class DecisionTreeRegressor:
    def __init__(self, min_samples_split=2, max_depth=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def fit(self, X, y):
        self.root = self._build_tree(X, y, depth=0)
        
    def _build_tree(self, X, y, depth = 0):
        node = TreeNode()
        
        if depth == self.max_depth or len(y) < self.min_samples_split:
            node.predicted_value = np.mean(y)
            return node

        best_feature, best_threshold, best_rss = self._find_best_split(X, y)

        if best_feature is None:
            node.predicted_value = np.mean(y)
            return node

        node.split_feature = best_feature
        node.threshold = best_threshold

        left_indices = X[:, best_feature] <= best_threshold
        right_indices = X[:, best_feature] > best_threshold

        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        node.left = self._build_tree(X_left, y_left, depth + 1)
        node.right = self._build_tree(X_right, y_right, depth + 1)

        return node

    def _find_best_split(self, X, y):
    
        best_rss = float('inf')
        best_feature = None
        best_threshold = None
    
        for feature_index in range(X.shape[1]):
            unique_threshold = sorted(set(X[:, feature_index]))
    
            for threshold in unique_threshold:
                left_indices = X[:, feature_index] <= threshold
                right_indices = X[:, feature_index] > threshold
    
                if left_indices.sum() == 0 or right_indices.sum() == 0:
                    continue
    
                y_left = y[left_indices]
                y_right = y[right_indices]
                
                current_rss = self. _calculate_rss(y_left, y_right)
    
                if current_rss < best_rss:
                    best_rss = current_rss
                    best_feature = feature_index
                    best_threshold = threshold
    
        return best_feature, best_threshold, best_rss

    def _calculate_rss(self, y_left, y_right):
        rss_left = np.sum((y_left - np.mean(y_left)) ** 2)
        rss_right = np.sum((y_right - np.mean(y_right)) ** 2)
        return rss_left + rss_right

    def predict(self, X):
        return np.array([self._predict_sample(sample, self.root) for sample in X])

    def _predict_sample(self, sample, node):
        if node.left is None and node.right is None:
            return node.predicted_value

        if sample[node.split_feature] <= node.threshold:
            return self._predict_sample(sample, node.left)
        else:
            return self._predict_sample(sample, node.right)
                

In [96]:
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tree_regressor = DecisionTreeRegressor(max_depth=5, min_samples_split=5)
tree_regressor.fit(X_train, y_train)

y_pred = tree_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error (MSE): {mse}")


Mean Squared Error (MSE): 0.0


In [97]:
print("Predictions: ", y_pred)
print("Actual: ", y_test)

Predictions:  [1. 0. 2. 1. 1. 0. 1. 2. 1. 1. 2. 0. 0. 0. 0. 1. 2. 1. 1. 2. 0. 2. 0. 2.
 2. 2. 2. 2. 0. 0.]
Actual:  [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
